* http://statline.cbs.nl/Statweb/publication/?DM=SLNL&PA=70703ned&D1=0&D2=a&HDR=T&STB=G1&VW=D
* http://statbel.fgov.be/nl/statistieken/opendata/datasets/bevolking/

In [1]:
import pandas as pd
import pymc3 as pm

%pylab inline

Populating the interactive namespace from numpy and matplotlib


* Worden er meer baby's geboren als de economie aantrekt?
* Hoe sterk is de clustering?

In [138]:
daily_ned = pd.read_csv("./data/Bevolkingsontwikkeli_171217121849.csv", encoding="latin-1", delimiter=";").reset_index()
daily_ned.columns = ["datum", "geboortes"]
daily_ned.geboortes = pd.to_numeric(daily_ned.geboortes, errors = "coerce")
daily_ned.head(7)

,datum,geboortes
0,Onderwerpen,NaN
1,Onderwerpen,NaN
2,Perioden,NaN
3,Totaal 1995,190513.0
4,Totaal januari 1995,16436.0
5,Zondag 1 januari 1995,368.0
6,Maandag 2 januari 1995,439.0


In [139]:
daily_ned = daily_ned[~daily_ned.datum.str.lower().str.contains("totaal")].dropna()
daily_ned.geboortes = daily_ned.geboortes.astype(int)

daily_ned.sample(10)

,datum,geboortes
5970,Woensdag 30-6-2010,543
4282,Woensdag 8 februari 2006,530
5802,Maandag 18-1-2010,510
1489,Zondag 15 november 1998,415
7980,2015 vrijdag 25 september,579
7544,2014 woensdag 6 augustus,553
7946,2015 zaterdag 22 augustus,445
5310,Woensdag 15 oktober 2008,567
5896,Maandag 19-4-2010,521
3564,Woensdag 31 maart 2004,598


In [141]:
fmt_yd = daily_ned.datum.str.extract("(?P<year>\d{4}) [a-z]{6,10} (?P<day>\d{1,2}) (?P<month_named>[a-z]{3,11})", expand = True).dropna()
fmt_yd[fmt_yd.any(axis = 1)].sample(5)

,year,day,month_named
7239,2013,26,oktober
8038,2015,22,november
6314,2011,21,mei
6422,2011,2,september
7790,2015,19,maart


In [142]:
months = fmt_yd.month_named.dropna().unique()

In [144]:
fmt_yd["month"] = fmt_yd.dropna().month_named.astype('category').cat.reorder_categories(months).cat.codes + 1
fmt_yd.sample(5)

,year,day,month_named,month
7560,2014,22,augustus,8
6818,2012,13,september,9
6808,2012,3,september,9
6988,2013,17,februari,2
6416,2011,28,augustus,8


In an undocumented feature that I found through [GitHub](https://github.com/nps/pandas/commit/cb7cdaa20ef5af421344819232ee289b93e22e7f), it is possible to combine columns of parts of dates into a datetime64-series. 

In [143]:
yd_dates = pd.to_datetime(fmt_yd.drop('month_named', axis = 1))
yd_dates.sample(5)

6511   2011-11-28
7108   2013-06-17
7786   2015-03-15
7119   2013-06-28
7187   2013-09-04
dtype: datetime64[ns]

In [133]:
fmt_m = daily_ned.datum.str.extract("(?P<day>\d{1,2}) (?P<month_named>[a-z]{3,11}) ?(?P<year>\d{4})", expand = True).dropna()
fmt_m["month"] = fmt_m.month_named.astype('category').cat.reorder_categories(months).cat.codes + 1
m_dates = pd.to_datetime(fmt_m.drop('month_named', axis = 1))

m_dates.sample(5)

4382   2006-05-16
4031   2005-06-18
2165   2000-08-15
2475   2001-06-03
1975   2000-02-13
dtype: datetime64[ns]

In [134]:
num_dates = pd.to_datetime(daily_ned.datum, format = "%d-%m-%Y", exact = False, errors = "coerce").dropna()
num_dates.sample(5)

5753   2009-12-10
5976   2010-07-05
5442   2009-02-12
5582   2009-06-28
5809   2010-01-25
Name: datum, dtype: datetime64[ns]

In [147]:
daily_ned = daily_ned.assign(date = pd.concat([yd_dates, m_dates, num_dates]))
daily_ned.sample(5)

,datum,geboortes,date
4888,Maandag 10 september 2007,563,2007-09-10
2921,Donderdag 1 augustus 2002,597,2002-08-01
916,Zaterdag 17 mei 1997,493,1997-05-17
917,Zondag 18 mei 1997,471,1997-05-18
6425,2011 maandag 5 september,518,2011-09-05


In [148]:
daily_ned[daily_ned.date.isnull()]

,datum,geboortes,date
5783,Overig,243,NaT
6553,Overig,171,NaT
6940,Overig,131,NaT
7314,Overig,144,NaT
7700,Overig,229,NaT
8086,Overig,121,NaT


In [152]:
daily_ned.dropna().date.dt.year.value_counts().sort_index().to_frame()

,date
1995,365
1996,366
1997,365
1998,365
1999,365
2000,366
2001,365
2002,365
2003,365
2004,366


Looks good!